In [2]:
import yfinance as yf

## Problem 1: Date from yfinance


In [8]:
# writing a function called get_data() to fetch data from the past 5 days for the FAANG stocks

faang_tickers = yf.Tickers('meta aapl amzn nflx goog') # reference: https://ranaroussi.github.io/yfinance/reference/yfinance.ticker_tickers.html
data = yf.download(faang_tickers, period='5mo')
print(data)


C:\Users\fmtie\AppData\Local\Temp\ipykernel_23528\1838444290.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(faang_tickers, period='5mo')


AttributeError: 'Tickers' object has no attribute 'replace'